In [7]:
import numpy as np

# \输入参数（先用假设值，拿到Bloomberg后直接替换即可）
usd_receivable = 500000
spot = 7.1185               # 即期汇率 (USDCNY)
usd_rate = 0.03845           # 美元3M利率 (proxy)
cny_rate = 0.015            # 人民币3M利率
T = 0.25

# Option market data (假设)
K = 7.05                    # 行权价
put_premium = 0.0265        # Put option premium (CNY per USD)

# Forward Hedge
F = spot * np.exp((cny_rate - usd_rate) * T)
rmb_forward = usd_receivable * F

# Option Hedge
premium_cost = put_premium * usd_receivable

future_spots = [6.8, 7.0, 7.1, 7.2, 7.4]
option_results = []
for ST in future_spots:
    payoff = max(K - ST, 0) * usd_receivable
    rmb_option = usd_receivable * ST + payoff - premium_cost
    option_results.append([ST, rmb_option/1e6])


print(f"Forward rate USD/CNY: {F:.4f}")
print(f"Forward hedge RMB (mn): {rmb_forward/1e6:.3f}\n")

print("Option hedge results (mn RMB):")
for ST, rmb in option_results:
    print(f"  If ST={ST:.2f}, Final RMB={rmb:.3f}")


Forward rate USD/CNY: 7.0769
Forward hedge RMB (mn): 3.538

Option hedge results (mn RMB):
  If ST=6.80, Final RMB=3.512
  If ST=7.00, Final RMB=3.512
  If ST=7.10, Final RMB=3.537
  If ST=7.20, Final RMB=3.587
  If ST=7.40, Final RMB=3.687


In [10]:
import numpy as np
from scipy.stats import norm

def fx_put_bs(S0, K, r_usd, r_cny, sigma, T):
    """
    Black–Scholes 外汇版本定价公式 (USD put = CNY call)
    """
    d1 = (np.log(S0 / K) + (r_cny - r_usd + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    P = K * np.exp(-r_cny * T) * norm.cdf(-d2) - S0 * np.exp(-r_usd * T) * norm.cdf(-d1)
    return P, d1, d2

# 假设参数（你可以用彭博查到的数据替换）
S0 = 7.1185     # 即期 USD/CNY
K = 7.10        # 执行价
r_usd = 0.03855  # 美元3M利率
r_cny = 0.015   # 人民币3M利率
sigma = 0.032   # 隐含波动率 ~3.2%
T = 0.25        # 到期时间 (年)

premium_bs, d1, d2 = fx_put_bs(S0, K, r_usd, r_cny, sigma, T)

# Bloomberg 给的市场 premium
premium_mkt = 0.02659  # 例如你截图的 Put Option Price

print(f"Black–Scholes theoretical price: {premium_bs:.5f} (CNY per USD)")
print(f"Bloomberg market premium:        {premium_mkt:.5f} (CNY per USD)")
print(f"d1={d1:.3f}, d2={d2:.3f}")


Black–Scholes theoretical price: 0.05762 (CNY per USD)
Bloomberg market premium:        0.02659 (CNY per USD)
d1=-0.197, d2=-0.213
